In [23]:
import os
import pickle
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgba2rgb, gray2rgb
import threading
import logging

In [24]:
def classify_image_with_unknown(image_path, model, threshold=0.7):
   
    img = imread(image_path)
    ##RGB conversion
    if len(img.shape) == 2:
        img = gray2rgb(img)
    elif img.shape[-1] == 4:
        img = rgba2rgb(img)

    #Prediction
    img = resize(img, img_size, anti_aliasing=True).flatten()
    probabilities = model.predict_proba(img.reshape(1, -1))
    max_confidence = np.max(probabilities)
    
    return model.predict(img.reshape(1, -1))[0] if max_confidence >= threshold else "unknown"

In [25]:
def test(test_image_path, test_results, best_estimator):
    if not os.path.exists(test_image_path):
        print(f"Error: The directory '{test_image_path}' does not exist.")
        exit()

    for file in os.listdir(test_image_path):
        image_path = os.path.join(test_image_path, file)
        if os.path.isfile(image_path) and file.endswith(('.png', '.jpg', '.jpeg')):
            # Classify the image
            result = classify_image_with_unknown(image_path, best_estimator, threshold=0.7)
            test_results[file] = result
            print(f"Image: {file}, Classification Result: {result}")
        else:
            print(f"Skipping {file}: Not a valid image file.")

In [26]:
from flask import Flask, jsonify
import time

app = Flask(__name__)
start_time = time.time()

In [27]:
# Loading the trained model
img_size = (15, 15)

model_file_name = r"E:\Education\Projects\Machine Learning\Computer Vision\Malicious-Sign-Detection\model\Models\classifier"
with open(model_file_name, 'rb') as file:
    best_estimator = pickle.load(file)

test_image_path = r"E:\Education\Projects\Machine Learning\Computer Vision\Malicious-Sign-Detection\traffic_light_data\test"
test_results = {}

test(test_image_path, test_results, best_estimator)

end_time = time.time()

Image: traffic_light_frame10092.png, Classification Result: unknown
Image: traffic_light_frame10527.png, Classification Result: unknown
Image: traffic_light_frame1102.png, Classification Result: unknown
Image: traffic_light_frame1131.png, Classification Result: unknown
Image: traffic_light_frame1160.png, Classification Result: red
Image: traffic_light_frame1189.png, Classification Result: red
Image: traffic_light_frame11890.png, Classification Result: unknown
Image: traffic_light_frame12035.png, Classification Result: unknown
Image: traffic_light_frame12064.png, Classification Result: red
Image: traffic_light_frame12093.png, Classification Result: unknown
Image: traffic_light_frame12122.png, Classification Result: unknown
Image: traffic_light_frame12151.png, Classification Result: red
Image: traffic_light_frame1218.png, Classification Result: red
Image: traffic_light_frame12180.png, Classification Result: red
Image: traffic_light_frame12209.png, Classification Result: red
Image: traffi

In [28]:
#from Scripts.train import y_val, y_pred, accuracy
from sklearn.metrics import accuracy_score, classification_report

# Loading calssification_report
with open('Scripts/Resources/classification_report', 'rb') as f:
    report = pickle.load(f)
    
execution_time = end_time - start_time
precision_macro = report['macro avg']['precision']
recall_macro = report['macro avg']['recall']
f1_score_macro = report['macro avg']['f1-score']
support_macro = report['macro avg']['support']

In [29]:
metrics = {
    #"Classification_Result": result,
    #"Validation_Accuracy": accuracy,
    "Execution_Time_of_Prediction": execution_time,
    "Macro_Precision_of_Training": precision_macro,
    "Macro_Recall_of_Training": recall_macro,
    "Macro_F1_score_of_Training": f1_score_macro,
    "Macro_Support_of_Training": support_macro,
    "Test_Results" : test_results
}

In [30]:
def run_flask():
    app.run(debug=False, port=5011)

In [31]:
@app.route('/data', methods=['GET'])
def get_data():
    return jsonify(metrics), 200

In [32]:
threading.Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off
